<a href="https://colab.research.google.com/github/romulokps/vacinasCOVID/blob/main/vacinasToledo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Elasticsearch

In [4]:
from datetime import date
import os
caminho = '/content/drive/My Drive/dados/apivacinas/vacinas/'
from elasticsearch import Elasticsearch, helpers
from joblib import Parallel, delayed
import pandas as pd
 
def write_row(res_list, csv_file, i, buffer_size):
    res_df = pd.DataFrame(res_list)
    if i <= buffer_size:
        res_df.to_csv(caminho + csv_file)
    else:
        res_df.to_csv(caminho + csv_file, mode='a', header=False)

def extract():
    es = Elasticsearch('https://imunizacao-es.saude.gov.br/',
                    http_auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'))
 
    index = 'desc-imunizacao'
    
    res_scan = helpers.scan(es, index = index,query={"query": {"match": {"estabelecimento_municipio_codigo": 412770}}}, scroll = '30m', size = 10000)
    csv_file_name = 'vacinas{}.csv.gz'.format(str(date.today()))
    csv_file_name = 'toledo{}.csv.gz'.format(str(date.today()))
    print("Starting to extract data")
    
    buffer_size = 1000000
    res_list = []
    i = 0
    for result in res_scan:
        res_list.append(result['_source'])
        i += 1
        if i % buffer_size == 0:
            write_row(res_list, csv_file_name, i, buffer_size)
            print("Written {} rows".format(i))
            res_list = []
    write_row(res_list, csv_file_name, i, buffer_size)  
    print("Finished to extract data for vacinas. Total rows: {}".format(i))
 

extract()


Starting to extract data
Finished to extract data for vacinas. Total rows: 3349


In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dados/apivacinas/vacinas/toledo2021-02-28.csv.gz')
df

,Unnamed: 0,vacina_categoria_codigo,vacina_nome,vacina_lote,estabelecimento_razaoSocial,estalecimento_noFantasia,paciente_endereco_cep,estabelecimento_uf,sistema_origem,paciente_endereco_nmMunicipio,paciente_racaCor_valor,paciente_dataNascimento,vacina_fabricante_nome,estabelecimento_municipio_nome,paciente_idade,vacina_grupoAtendimento_codigo,vacina_descricao_dose,paciente_nacionalidade_enumNacionalidade,vacina_codigo,estabelecimento_municipio_codigo,@version,paciente_id,vacina_categoria_nome,vacina_dataAplicacao,document_id,@timestamp,paciente_enumSexoBiologico,estabelecimento_valor,paciente_endereco_uf,paciente_endereco_coPais,paciente_endereco_nmPais,paciente_endereco_coIbgeMunicipio,vacina_grupoAtendimento_nome,paciente_racaCor_codigo,vacina_fabricante_referencia,data_importacao_rnds
0,0,2,Covid-19-Coronavac-Sinovac/Butantan,210011A,PREFEITURA DO MUNICIPIO DE TOLEDO,ESFSB JARDIM EUROPA,85907,PR,SIGSS MV,TOLEDO,PARDA,1926-02-25,Covid-19 - Coronavac,TOLEDO,94,205,1ª Dose,B,86,412770,1,907ed926664edaa371b4c957c8c7ce1e0fd93a140fc14c...,Faixa Etária,2021-02-10T00:00:00.000Z,be3cefb4-d718-4c20-8916-462ae8f17ef0-i0b0,2021-02-24T16:21:26.928Z,F,4056868,PR,10,BRASIL,412770,Pessoas de 80 anos ou mais,3,NaN,NaN
1,1,9,Vacina Covid-19 - Covishield,4120Z005,PREFEITURA DO MUNICIPIO DE TOLEDO,EAP JARDIM COOPAGRO,85903,PR,Novo PNI,TOLEDO,SEM INFORMACAO,1982-10-15,FUNDACAO OSWALDO CRUZ,TOLEDO,38,909,1ª Dose,B,85,412770,1,3f515a555db20f681b9f5fd70e3c96aca9259dc6734ce4...,Trabalhadores de Saúde,2021-01-26T03:00:00.000Z,a8c88767-fb1e-48f2-9fd9-d0ae909edce2-i0b0,2021-02-24T16:27:06.249Z,F,4056841,PR,10,BRASIL,412770,Fisioterapeutas,99,Organization/33781055000135,NaN
2,2,9,Vacina Covid-19 - Covishield,4120Z005,PREFEITURA DO MUNICIPIO DE TOLEDO,EAP JARDIM COOPAGRO,85910,PR,Novo PNI,TOLEDO,BRANCA,1973-11-26,FUNDACAO OSWALDO CRUZ,TOLEDO,47,926,1ª Dose,B,85,412770,1,513b5745839f4721a179425c89aa8f95109996724fa905...,Trabalhadores de Saúde,2021-01-29T03:00:00.000Z,aae9e6b7-f9e6-48f1-b002-145451a950ec-i0b0,2021-02-24T16:12:06.718Z,F,4056841,PR,10,BRASIL,412770,Outros,1,Organization/33781055000135,NaN
3,3,9,Covid-19-Coronavac-Sinovac/Butantan,202010018,PREFEITURA DO MUNICIPIO DE TOLEDO,EAP JARDIM COOPAGRO,,PR,Novo PNI,PALOTINA,BRANCA,1990-04-24,FUNDACAO BUTANTAN,TOLEDO,30,907,1ª Dose,B,86,412770,1,beac52c8277ae3ca012c2ee7637b3d0fce5b3258549142...,Trabalhadores de Saúde,2021-01-21T00:00:00.000Z,87253b39-89c0-4cc2-9027-e76ce5858b7b-i0b0,2021-02-24T16:46:16.222Z,F,4056841,PR,10,BRASIL,411790,Enfermeiro(a),1,Organization/61189445000156,NaN
4,4,9,Vacina Covid-19 - Covishield,4120Z005,PREFEITURA DO MUNICIPIO DE TOLEDO,EAP JARDIM COOPAGRO,,PR,Novo PNI,TOLEDO,SEM INFORMACAO,1982-03-19,FUNDACAO OSWALDO CRUZ,TOLEDO,38,914,1ª Dose,B,85,412770,1,5327d0bd2aa83841899d42c340ffc6c0c222d3cfe1bbe4...,Trabalhadores de Saúde,2021-01-29T03:00:00.000Z,5b56019c-38d3-47b2-a529-f2c43ba0caf7-i0b0,2021-02-24T16:01:30.814Z,M,4056841,PR,10,BRASIL,412770,Motorista de Ambulância,99,Organization/33781055000135,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3344,3344,9,Covid-19-Coronavac-Sinovac/Butantan,202010018,PREFEITURA MUNICIPAL DE TOLEDO,ESFSB JARDIM PANORAMA,85903,PR,SIGSS MV,TOLEDO,BRANCA,1993-01-14,Covid-19 - Coronavac,TOLEDO,28,923,2ª Dose,B,86,412770,1,cdcf28ce4cc1bcb7f755518a3675be9ae4cae1dc15a680...,Trabalhadores de Saúde,2021-02-17T00:00:00.000Z,39d909f1-ed8b-498a-a51f-1fc6ec30572e-i0b0,2021-02-26T17:21:18.221Z,M,4056981,PR,10,BRASIL,412770,Técnico de Enfermagem,1,NaN,2021-02-17 18:36:39
3345,3345,9,Vacina Covid-19 - Covishield,4120Z005,PREFEITURA DO MUNICIPIO DE TOLEDO,EAP JARDIM COOPAGRO,85902,PR,Novo PNI,TOLEDO,BRANCA,1994-12-30,FUNDACAO OSWALDO CRUZ,TOLEDO,26,923,1ª Dose,B,85,412770,1,69dfdf19c49e57f2462811d7e6503fc808c168efce4c28...,Trabalhadores de Saúde,2021-01-29T03:00:00.000Z,a9edabf6-fc82-4bd4-aa02-50928fa84e61-i0b0,2021-02-26T17:21:38.708Z,F,4056841,PR,10,